3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?